In [3]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model

In [8]:
model = load_model('model.h5')

with open ('lable_encoder.pkl' , 'rb') as file:
    lable_encoder = pickle.load(file)

with open ('one_hot_encoding.pkl' , 'rb') as file:
    one_hot = pickle.load(file) 

with open ('standardscaler.pkl' , 'rb') as file:   
    scaler = pickle.load(file)    

In [6]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [9]:
# One-hot encode 'Geography'
geo_encoded = one_hot.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot.get_feature_names_out(['Geography']))
geo_encoded_df


C:\Users\91772\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [30]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [31]:
input_df['Gender'] = lable_encoder.transform(input_df['Gender'])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [32]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [34]:
input_scale = scaler.transform(input_df)
input_scale

array([[-0.51222865,  0.90636285,  0.10591292, -0.69703024, -0.26147196,
         0.8016426 ,  0.64900815,  0.97238125, -0.86754814,  0.99575899,
        -0.57812007, -0.57330877]])

In [35]:
prediction = model.predict(input_scale)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


array([[0.02963436]], dtype=float32)

In [37]:
predict_prob = prediction[0][0]
predict_prob

0.029634358

In [38]:
if predict_prob > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
